In [1]:
import re
import pandas as pd

In [2]:
file = open(file="U.S. Patents.txt", mode='r') #opening the file in read mode
file_content_raw = file.read()
file.close()
text1=re.compile("<\?xml version=\"1\.0\" encoding\=\"UTF\-8\"\?>")
file_content = text1.split(file_content_raw)
while '' in file_content:
    file_content.remove('')
print("No of patents :", len(file_content))    

No of patents : 8119


In [3]:
# Writing regular expressions for the desired columns

grant_id = re.compile('file\=\"([U][S]\w\w\d{6})\-\d{8}\.XML\"')   
patent_title = re.compile("<invention-title id=\"\w{5,6}\">(.*?)</invention-title>") 
kind = re.compile("<kind>([A-Z]\d)</kind>")
number_of_claim = re.compile("\<number\-of\-claims\>(\d{1,4})\<\/number\-of\-claims\>")
first_name=re.compile("<first-name>(.*?)</first-name>")
last_name=re.compile("<last-name>(.*?)</last-name>")
citation_by_examiner = re.compile("\<category\>cited by examiner<\/category\>")
citation_by_applicant = re.compile("\<category>cited by applicant\<\/category\>")
claim_text=re.compile("<claim-text>[\s\S<]*</claim-text>")
abstract=re.compile("\<abstract id\=\"abstract\"\>\n\<p id\=\"p\-0001\" num\=\"0000\"\>(.*?)\<\/p\>\n\<\/abstract\>")

# some extra care with claim-text
cleaner = re.compile('<.*?>') 
cleaner2 = re.compile('\n')
cleaner3 = re.compile('\,\,\,')
cleaner4 = re.compile("[\.][\,][\,]")
cleaner5 = re.compile("[\,][\,]")
cleaner6 = re.compile("[\;][\,]")

In [4]:
gid_list, title_list, kind_list, no_of_claim_list, name_list, applicant_list, examiners_list, claim_list, abstract_list, = ([] for i in range(9))

for line in file_content:
    
    
    gid=grant_id.findall(line) #to find grant_id
    title=patent_title.findall(line) #to find patent_title
    kinds=kind.findall(line) #to find kind
    sclaim=number_of_claim.findall(line) #to find no_of_claims
    
    #to find inventors
    inventors=re.findall("<inventor.*?>[\s\S]*</inventor>",line)
    for person in inventors:
        first=first_name.findall(person)
        last=last_name.findall(person)
    name = [firstName +" "+ lastName for firstName, lastName in zip(first,last)]
    if len(name)==0:
        names="NA"
    else:
        names=name
    
    # here we count citation_by_applicant
    if len(citation_by_applicant.findall(line))==0:
        citation_by_applicants=0
    else:
        citation_by_applicants=len(citation_by_applicant.findall(line)) 
    
    # count for citation_by_examiner
    if len(citation_by_examiner.findall(line))==0:
        citation_by_examiners=0
    else:    
        citation_by_examiners=len(citation_by_examiner.findall(line))   
    
    # For claim_text
    if (len(re.findall("<claim-text>[\s\S<]*</claim-text>",line))==0):
        claim_text=["NA"]
    else:
        claim_text=re.findall("<claim-text>[\s\S<]*</claim-text>",line) 
    
    # For abstract
    abst=abstract.findall(line)
    if len(abst)==0:
        abstracts=["NA"]
    else:    
        abstracts=abst  
    
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(gid)!=0:                             
        gid_list.append(gid[0])
        title_list.append(title[0])
        kind_list.append(kinds[0])
        no_of_claim_list.append(sclaim[0])
        name_list.append(names)
        applicant_list.append(citation_by_applicants)
        examiners_list.append(citation_by_examiners)
        claim_list.append(claim_text[0])
        abstract_list.append(abstracts[0])
        
# cleaning claim text         
element=0
for items in claim_list:
    claim_list[element]=re.sub(cleaner,'',claim_list[element])
    claim_list[element]=re.sub(cleaner2,',',claim_list[element])
    claim_list[element]=re.sub(cleaner3,',',claim_list[element])
    claim_list[element]=re.sub(cleaner4,'.,',claim_list[element])
    claim_list[element]=re.sub(cleaner5,',',claim_list[element])
    claim_list[element]=re.sub(cleaner6,'; ',claim_list[element])
    element=element+1

# For kind 
Kind1 = [w.replace('P2', 'Plant Patent Grant(with a published application) issued on or after January 2, 2001') for w in kind_list]
Kind2 = [w.replace('B2', 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.') for w in Kind1]
Kind3 = [w.replace('S1', 'Design Patent') for w in Kind2]
Kind4 = [w.replace('B1', 'Utility Patent Grant (no published application) issued on or after January 2, 2001.') for w in Kind3]

In [5]:
# Creating data frame
data_frame = pd.DataFrame(
    {'grant_id': gid_list,
     'patent_title': title_list,
     'kind': Kind4,
     'number_of_claims':no_of_claim_list,
     'inventors':name_list,
     'citations_applicant_count':applicant_list,
     'citations_examiner_count':examiners_list,
     'claims_text':claim_list,
     'abstract':abstract_list
    })

In [6]:
data_frame.isnull().sum()

grant_id                     0
patent_title                 0
kind                         0
number_of_claims             0
inventors                    0
citations_applicant_count    0
citations_examiner_count     0
claims_text                  0
abstract                     0
dtype: int64

In [7]:
data_frame

,grant_id,patent_title,kind,number_of_claims,inventors,citations_applicant_count,citations_examiner_count,claims_text,abstract
0,USD0864516,Thin food cluster,Design Patent,1,"[Falguni Dharmendrakumar Chauhan, Maura E. Col...",242,18,"The ornamental design for a thin food cluster,...",NA
1,USD0864517,Chocolate bar,Design Patent,1,[Jerome Sli],0,14,I claim the ornamental design for a chocolate ...,NA
2,USD0864518,Golf glove with ventilation means,Design Patent,1,[Sammerjit Raikhy],0,14,The ornamental design for a golf glove with ve...,NA
3,USD0864519,Glove,Design Patent,1,"[Steven W. Hyma, Grant T. Squiers]",43,9,"We claim the ornamental design for a glove, as...",NA
4,USD0864520,Wearable towel with openings,Design Patent,1,[Erin Robertson],1,5,The ornamental design of a wearable towel with...,NA
5,USD0864521,Men's underwear,Design Patent,1,[Jamil Nacif Nacif Benavides],0,16,"An ornamental design for a men's underwear, as...",NA
6,USD0864522,Padded underwear,Design Patent,1,[Jamil Nacif Benavides],0,11,"An ornamental design for a padded underwear, a...",NA
7,USD0864523,Supportive garment,Design Patent,1,"[Craig H. Diamond, Alexander Coriano, III, Bri...",0,10,The ornamental design for a supportive garment...,NA
8,USD0864524,Jeans,Design Patent,1,"[Valerie Lyon, Ron Sheeley, Becky Taylor, Jerr...",0,11,"The ornamental design for jeans, as shown and ...",NA
9,USD0864525,Pants with contrasting waistband,Design Patent,1,"[Valerie Lyon, Ron Sheeley, Becky Taylor, Jerr...",0,11,The ornamental design for pants with contrasti...,NA


In [8]:
data_frame["inventors"][0]

['Falguni Dharmendrakumar Chauhan',
 'Maura E. Collins Titone',
 'Priya Kalwani',
 'Meredith Ann Keiling',
 'Daniel H. Levison',
 'Mihaelos N. Mihalos',
 'Janelle Marie Myers',
 'Nimesh K. Patel',
 'Timothy David Towers']

In [9]:
data_frame["claims_text"][0]

'The ornamental design for a thin food cluster, as shown and described.'

In [10]:
data_frame.to_csv("U.S. Patents.csv",index=False)